## Recordatorio: De Funciones a Clases


In [ ]:
# Recordatorio del notebook 04-oop_basics:
# Empezamos con variables separadas...
account1_holder = "Alice"
account1_balance = 1000.0

# ...luego dictionaries...
account1 = {"holder": "Alice", "balance": 1000.0}

# ...y finalmente clases
class BankAccount:
    def __init__(self, holder, balance):
        self.holder = holder
        self.balance = balance

Ahora vamos un paso más allá: **¿Cómo organizamos el código cuando crece?**

---

## Versión 1: Monolítica (Todo en un archivo)

### El Problema: Código que Crece

Empezamos con una clase simple de cuenta bancaria. Pero ahora necesitamos:
- ✅ Identificar cuentas por IBAN (no por nombre)
- ✅ Validar que el IBAN es correcto
- ✅ Controlar errores

**Primera aproximación**: poner todo en la clase.

In [ ]:
# Mira el archivo: code-examples/bank_evolution/v1_monolithic/bank.py

# Resumen de la estructura:
"""
class BankAccount:
    def __init__(self, iban, balance):
        if not self._is_valid_iban(iban):  # ← Validación dentro de la clase
            raise ValueError(...)
        self._iban = iban
        self._balance = balance
    
    def _is_valid_iban(self, iban):       # ← Método privado de validación
        pattern = r'^ES\d{22}$'
        return bool(re.match(pattern, iban))
    
    def deposit(self, amount): ...
    def withdraw(self, amount): ...
    def transfer(self, target, amount): ...
"""

print("✓ Código funcional pero todo mezclado")

✓ Código funcional pero todo mezclado


<>:13: SyntaxWarning: invalid escape sequence '\d'
<>:13: SyntaxWarning: invalid escape sequence '\d'
C:\Users\avidaldo\AppData\Local\Temp\ipykernel_15400\1799094423.py:13: SyntaxWarning: invalid escape sequence '\d'
  pattern = r'^ES\d{22}$'


### ¿Qué está mal?

| Problema | Explicación |
|----------|-------------|
| **Responsabilidades mezcladas** | La clase hace TODO: gestionar cuentas Y validar IBANs |
| **No reutilizable** | ¿Y si otro módulo necesita validar IBANs? |
| **Difícil de probar** | Para probar la validación, hay que crear una cuenta completa |
| **Difícil de extender** | ¿Y si necesitamos validar otros países? La clase crece y crece |

---

## Versión 2: Funcional (Extrayendo Funciones)

### Principio: **SoC** (Separation of Concerns)

> "Cada parte del código debería ocuparse de una sola cosa."

**Solución**: Extraer las validaciones a funciones independientes.

In [ ]:
# Mira el archivo: code-examples/bank_evolution/v2_functional/bank.py

# Ahora tenemos:
"""
# ---- FUNCIONES DE VALIDACIÓN (separadas) ----
def validate_iban_format(iban):
    pattern = r'^ES\d{22}$'
    return bool(re.match(pattern, iban))

def validate_positive_amount(amount):
    return amount > 0

# ---- CLASE (más simple, enfocada) ----
class BankAccount:
    def __init__(self, iban, balance):
        if not validate_iban_format(iban):  # ← Usa función externa
            raise ValueError(...)
        ...
    
    def deposit(self, amount):
        if not validate_positive_amount(amount):  # ← Usa función externa
            raise ValueError(...)
        ...
"""

print("✓ Mejor: responsabilidades separadas")
print("✓ Las funciones se pueden probar independientemente")

✓ Mejor: responsabilidades separadas
✓ Las funciones se pueden probar independientemente


<>:7: SyntaxWarning: invalid escape sequence '\d'
<>:7: SyntaxWarning: invalid escape sequence '\d'
C:\Users\avidaldo\AppData\Local\Temp\ipykernel_15400\182876172.py:7: SyntaxWarning: invalid escape sequence '\d'
  pattern = r'^ES\d{22}$'


### Mejoras de la v2

✅ **SoC**: Validación separada de lógica de cuenta  
✅ **Testeable**: Puedes probar `validate_iban_format("ES123")` directamente  
✅ **Reutilizable**: Otras clases pueden usar las mismas validaciones  

Pero todavía hay un problema: **todo sigue en un archivo**.

---

## Versión 3: Modular (Archivos Separados)

### Principio: **DRY** (Don't Repeat Yourself)

> "No repitas código. Crea componentes reutilizables."

### El Catalizador: Validación Compleja de IBAN

El IBAN no es solo un formato. Tiene un **checksum** (dígitos de control) calculado con el algoritmo **MOD-97**.

```
ES91 2100 0418 4502 0005 1332
  ^^
  Checksum - debe cumplir MOD-97
```

Implementar esto en `bank.py` haría el archivo demasiado largo y mezclado.

**Solución**: Crear un módulo separado `validators.py`

In [ ]:
# Mira los archivos: code-examples/bank_evolution/v3_modular/

# Estructura:
"""
v3_modular/
    validators.py    ← Módulo de validaciones (con MOD-97 completo)
    bank.py          ← Solo la lógica de cuenta bancaria
"""

# En bank.py:
"""
from validators import validate_iban, validate_positive_amount

class BankAccount:
    def __init__(self, iban, balance):
        if not validate_iban(iban):  # ← Validación completa (formato + checksum)
            raise ValueError(...)
"""

print("✓ Código organizado en módulos")
print("✓ validators.py se puede importar desde otros proyectos")

✓ Código organizado en módulos
✓ validators.py se puede importar desde otros proyectos


### El Algoritmo MOD-97 (Curiosidad)

Así se valida el checksum de un IBAN:

```python
def validate_iban_checksum(iban):
    # 1. Mover los 4 primeros caracteres al final
    #    ES9121000418... → 21000418...ES91
    rearranged = iban[4:] + iban[:4]
    
    # 2. Convertir letras a números (E=14, S=28)
    #    21000418...1428 91
    numeric = "".join(str(ord(c) - ord('A') + 10) if c.isalpha() else c 
                      for c in rearranged)
    
    # 3. El número debe dar resto 1 al dividir por 97
    return int(numeric) % 97 == 1
```

**Este código no pertenece a la clase `BankAccount`**. Pertenece a un módulo de validación.

### Mejoras de la v3

✅ **DRY**: El módulo `validators` se puede reutilizar en múltiples proyectos  
✅ **SoC**: Cada archivo tiene un propósito claro  
✅ **Mantenible**: Cambios en validación no afectan a `bank.py`  

Pero... ¿y si necesitamos más tipos de validaciones?

---

## Versión 4: Paquete (Estructura Profesional)

### Principio: **SRP** (Single Responsibility Principle)

> "Cada módulo/clase debería tener una sola razón para cambiar."

### El Problema con v3

Si añadimos más validaciones a `validators.py`:
- Validación de tarjetas de crédito
- Validación de DNI/NIE
- Validación de emails

El archivo `validators.py` se vuelve **enorme y difícil de navegar**.

**Solución**: Crear un **paquete** con submódulos.

In [ ]:
# Mira los archivos: code-examples/bank_evolution/v4_package/

# Estructura:
"""
v4_package/
    validators/              ← Paquete
        __init__.py          ← Define qué se exporta
        iban.py              ← Solo validaciones de IBAN
        amount.py            ← Solo validaciones de cantidades
    bank.py
"""

# En bank.py, la importación es igual:
"""
from validators import validate_iban, validate_positive_amount
# ↑ Funciona gracias a __init__.py
"""

print("✓ Arquitectura escalable")
print("✓ Cada submódulo tiene una responsabilidad única (SRP)")

✓ Arquitectura escalable
✓ Cada submódulo tiene una responsabilidad única (SRP)


### El Papel de `__init__.py`

```python
# validators/__init__.py

from .iban import validate_iban, validate_iban_format
from .amount import validate_positive_amount

__all__ = ['validate_iban', 'validate_iban_format', 'validate_positive_amount']
```

Esto permite:
```python
# En vez de:
from validators.iban import validate_iban
from validators.amount import validate_positive_amount

# Podemos escribir:
from validators import validate_iban, validate_positive_amount
```

**Importaciones más limpias** sin sacrificar organización.

### Mejoras de la v4

✅ **SRP**: Cada módulo hace una cosa  
✅ **Escalable**: Fácil añadir `validators/credit_card.py`, etc.  
✅ **Profesional**: Estructura usada en proyectos reales  
✅ **Importaciones limpias**: Gracias a `__init__.py`  

---

## Comparación Final: Las 4 Versiones

| Aspecto | v1 Monolítica | v2 Funcional | v3 Modular | v4 Paquete |
|---------|--------------|--------------|------------|------------|
| **Archivos** | 1 | 1 | 2 | 4+ |
| **Organización** | ❌ Todo mezclado | ⚠️ Funciones separadas | ✅ Módulos | ✅✅ Paquete |
| **Reutilización** | ❌ No | ⚠️ Limitada | ✅ Sí | ✅✅ Máxima |
| **Testeable** | ❌ Difícil | ✅ Funciones | ✅ Módulos | ✅✅ Aislado |
| **Escalabilidad** | ❌ Crece mal | ⚠️ Un archivo grande | ✅ Múltiples archivos | ✅✅ Submódulos |
| **Mantenibilidad** | ❌ Baja | ⚠️ Media | ✅ Alta | ✅✅ Muy alta |
| **Principios** | Ninguno | SoC básico | SoC + DRY | SoC + DRY + SRP |

---

## ¿Cuándo Usar Cada Versión?

### v1 - Monolítica
✅ **Usa cuando:**
- Prototipando rápido
- Script de una sola vez
- Menos de 100 líneas

❌ **Evita cuando:**
- El código crece más de 200 líneas
- Necesitas reutilizar lógica
- Trabajan varias personas

### v2 - Funcional
✅ **Usa cuando:**
- Separas lógica de negocio de utilidades
- Aún es un archivo manejable (<500 líneas)

❌ **Evita cuando:**
- Tienes muchas funciones de ayuda
- Quieres distribuir código

### v3 - Modular
✅ **Usa cuando:**
- Tienes lógica compleja y reutilizable
- Quieres separar responsabilidades
- El proyecto tiene múltiples archivos

❌ **Evita cuando:**
- Un módulo crece demasiado (>1000 líneas)
- Necesitas submódulos temáticos

### v4 - Paquete
✅ **Usa cuando:**
- Proyecto grande con muchas responsabilidades
- Necesitas jerarquía de módulos
- Código para distribuir (librería)

❌ **Evita cuando:**
- Proyecto muy simple
- Sobre-ingeniería para algo pequeño

---

## Principios de Diseño: Resumen

### 🔄 DRY (Don't Repeat Yourself)

**Mal:**
```python
# Validación copiada en 3 lugares
if not re.match(r'^ES\d{22}$', iban1): ...
if not re.match(r'^ES\d{22}$', iban2): ...
if not re.match(r'^ES\d{22}$', iban3): ...
```

**Bien:**
```python
def validate_iban(iban): ...
if not validate_iban(iban1): ...
if not validate_iban(iban2): ...
```

### 🎯 SoC (Separation of Concerns)

**Mal:**
```python
class BankAccount:
    def __init__(self, iban):
        # ❌ La clase hace TODO: gestionar cuentas Y validar
        if not self._validate_iban_format(iban): ...
        if not self._validate_iban_checksum(iban): ...
```

**Bien:**
```python
# Validación en su propio módulo
from validators import validate_iban

class BankAccount:
    def __init__(self, iban):
        # ✅ La clase solo gestiona cuentas
        if not validate_iban(iban): ...
```

### 📦 SRP (Single Responsibility Principle)

**Mal:**
```python
# validators.py - hace DEMASIADO
def validate_iban(): ...
def validate_credit_card(): ...
def validate_dni(): ...
def validate_email(): ...
# ... 50 funciones más
```

**Bien:**
```python
validators/
    iban.py         # Solo IBANs
    credit_card.py  # Solo tarjetas
    dni.py          # Solo DNI/NIE
```

---

## Ejercicio Práctico

### Ejercicio 1: Identifica el Problema

Mira este código y di qué principio viola:

In [ ]:
class User:
    def __init__(self, email, password):
        # Validación de email inline
        if "@" not in email or "." not in email.split("@")[1]:
            raise ValueError("Email inválido")
        
        # Validación de contraseña inline
        if len(password) < 8:
            raise ValueError("Contraseña muy corta")
        if not any(c.isupper() for c in password):
            raise ValueError("Contraseña debe tener mayúscula")
        
        self.email = email
        self.password = password

# ¿Qué principio se viola? ¿Cómo lo mejorarías?

### Ejercicio 2: Refactoriza

Mejora el código anterior aplicando **v2** (funcional):

In [ ]:
# Tu solución aquí

def validate_email(email):
    # TODO: implementar
    pass

def validate_password(password):
    # TODO: implementar
    pass

class User:
    def __init__(self, email, password):
        # TODO: usar las funciones de validación
        pass

---

## Conclusiones

### El Camino de la Evolución del Código

```
Monolítico  →  Funcional  →  Modular  →  Paquete
(v1)           (v2)         (v3)        (v4)

Rápido      →  Organizado →  Reutilizable → Escalable
```

### Reglas de Oro

1. **Empieza simple** (v1), pero reconoce cuándo refactorizar
2. **Si copias código**, crea una función (DRY)
3. **Si una clase hace demasiado**, extrae funciones (SoC)
4. **Si un archivo es muy largo**, crea módulos (SRP)
5. **Si un módulo hace muchas cosas**, crea un paquete (SRP)

### Señales de que Necesitas Refactorizar

🚨 **Archivo >500 líneas**  
🚨 **Copias de código** (mismo código en varios lugares)  
🚨 **Difícil de probar** (necesitas mucho setup)  
🚨 **Difícil de explicar** qué hace una clase/función  
🚨 **Merge conflicts** frecuentes (todos editan el mismo archivo)  

### Próximos Pasos

- Explora los archivos en [code-examples/bank_evolution/](../code-examples/bank_evolution/)
- Ejecuta cada versión y compara
- Aplica estos principios en tus propios proyectos

**Recuerda**: La arquitectura perfecta no existe al empezar. Evoluciona con tu código. 🚀